# Association Rule Mining in Mass-Spectrometry Data
### *Muhammad Haseeb and Muhammad Usman Tariq*
##### *CAP6778: Adv. Topics in Data Mining - Fall 2019*
##### *School of Computing and Information Sciences (SCIS)*
##### *Florida International University (FIU)*

### The MIT License

##### *Copyright 2019 Muhammad Haseeb and Muhammad Usman Tariq*

Permission is hereby granted, free of charge, to any person obtaining a copy of this software 
and associated documentation files (the "Software"), to deal in the Software without restriction, 
including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, 
and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, 
subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial 
portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT 
LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. 
IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, 
WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE 
SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

## Required Imports

In [1]:
# Import the required libraries
import os
import re
import sys
import math
import time
import urllib
import operator
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from IPython.display import clear_output

## Data Acquisition
Let's get the annotated HUMAN MS/MS spectral data libraries from <a href="https://chemdata.nist.gov/dokuwiki/doku.php?id=peptidew:cdownload" class="external">NIST.</a> Note that this can be done for any spectral library or multiple spectral libraries at once as well.

In [2]:
mouse = 'cptac2_mouse_hcd_selected.msp.tar.gz'
msp_file = './data/cptac2_mouse_hcd_selected.msp'
data_dir = os.path.expanduser("./data")

if not os.path.isdir(data_dir):
    os.mkdir(data_dir)

if not os.path.exists(msp_file):
    if not os.path.exists(mouse):
        !wget 'ftp://chemdata.nist.gov/download/peptide_library/libraries/cptaclib/2015/cptac2_mouse_hcd_selected.msp.tar.gz'
    
    # Extract the tar ball into MSP file and move into the speclib folder
    !tar xvf 'cptac2_mouse_hcd_selected.msp.tar.gz'
    os.rename('./cptac2_mouse_hcd_selected.msp', msp_file)

### Data Extraction Hyperparameters

In [3]:
# Data extraction Hyperparamters
divider = 5                     # Divide every m/z by this number.
max_len = int(5000 / divider)  # Max allowed m/z value.
filter_peaks = 100              # How many peaks to keep after filtration.

In [4]:
'''To keep updating this dict for multiple files. Dont define it again.'''
pep_spec = {}

### MSP File Parser

Read the MSP file, extract the required ions and pre-prcocess spectra

In [5]:
f=open(msp_file, "r")
lines = f.readlines()
f.close()

isName = isMW = isNumPeaks = False
new = prev = 0
i = 0


while i < len(lines):
    '''
    Process each line of the file.
    '''
    line = lines[i]
    i += 1
    
    splits = line.split(':')
    
    '''Keep going through lines. Name line is peptide and
    all the lines after Num peaks are the spectrum'''
    if splits[0] == 'Name':
        split1 = splits[1]
        pep = split1.split('/')[0].lstrip(' ')
        isName = True
        
    if isName and splits[0] == 'MW':
        mass = float(splits[1]) # Not using it anywhere. It's just there.
        isMW = True
        
    if isName and isMW and splits[0] == 'Num peaks':
        '''After this line, we will start processing the spectrum.'''
        numPeaks = int(splits[1]) # Just in case.
        
        temp_spec = np.zeros(max_len)
        while (lines[i] != '\n'):
            mzline = lines[i]
            i += 1
            mzsplits = mzline.split('\t')
            # Get the m/z and intenisty value of each line.
            moz, intensity, pattern = float(mzsplits[0]), float(mzsplits[1]), mzsplits[2]

            # Filter out only the non-noisy terminal ions from the spectrum
            lbls = re.findall(r"(?<!\/)(?<![A-Z])([abcxyz][0-9]+)", pattern)

            if lbls:
                # Dividing m/z by the divider.
                temp_spec[round(moz / divider)] = intensity

        isNumPeaks = True
        
    if isName and isMW and isNumPeaks:
        '''At this point, we are done reading one spectrum.
        Place everything where it needs to go and read the next one.'''
        isName = isMW = isNumPeaks = False
        
        # This is the filter. Gets the top 100 peaks
        # and returns them in one hot encoded vector of size max_len
        top_indx = np.argpartition(temp_spec, -filter_peaks)[-filter_peaks:]
        spec = np.zeros(max_len, dtype=bool)
        spec[top_indx] = True
        
        # Creating dictionary. Key: peptide, Value: spectrum.
        # As a side effect we'll get rid of duplicates.
        if pep not in pep_spec: pep_spec[pep] = spec
        
        # Displays the progress.
        new = int((i/len(lines)) * 100)
        if new > prev:
            clear_output(wait=True)
            print(str(new) + '%')
            prev = new


99%


## Frequent Itemset Mining
Construct the market-basket dataframe.

### Frequent Itemset Hyperparameters

In [6]:
max_itemset_size = 4     # Max size of the frequent itemsets.
min_support = 0.15       # Maybe we should increase this?

### DataFrame Construction

In [7]:
# Just get the value from the dict.
buckets = list(pep_spec.values())
print(len(buckets))

# The apriori function needs a dataframe.
df_buckets = pd.DataFrame(buckets, columns=list(range(0, max_len)))

10026


In [9]:
df_buckets

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


### Apriori Algorithm
Using the association rule mining algorithms from: *Agrawal, Rakesh, and Ramakrishnan Srikant. "Fast algorithms for mining association rules." Proc. 20th int. conf. very large data bases, VLDB. Vol. 1215. 1994*. Using implementation from the [mlxtend.apriori](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/) Python package.

In [10]:
freq_itemsets = apriori(df_buckets,
                        min_support=min_support, 
                        max_len=max_itemset_size, 
                        verbose=1,
                        low_memory=True)

Iteration: 3969517 | Sampling itemset size 4


### Freq Itemsets Visualization

Now that we have mined the frequent itemsets from the dataframe, let's visualize some stats and see if we need to refine any hyperparameters and mine itemsets again.

In [38]:
# Add another column to store lengths of the frequent itemsets
freq_itemsets['length'] = freq_itemsets['itemsets'].apply(lambda x: len(x))

# Filter out the small itemsets (of length < 3)
filtered_freqITS = freq_itemsets[ (freq_itemsets['length'] > 2) &
                                        (freq_itemsets['support'] > 0.8) ]
# We found 625 filtered frequent itemsets
disp_ffITS = filtered_freqITS.reset_index().rename(columns={'index' : 'itemset#'})

filtered_freqITS = filtered_freqITS.drop(['length'], axis=1)

In [39]:
disp_ffITS

,itemset#,support,itemsets,length
0,159413,0.804608,"(352, 353, 351)",3
1,159414,0.803212,"(352, 354, 351)",3
2,159442,0.803511,"(353, 354, 351)",3
3,159443,0.800020,"(353, 355, 351)",3
4,159470,0.800319,"(354, 355, 351)",3
5,159845,0.809595,"(352, 353, 354)",3
6,159846,0.805506,"(352, 353, 355)",3
7,159847,0.803012,"(352, 353, 356)",3
8,159848,0.801217,"(352, 353, 357)",3
9,159849,0.800818,"(352, 353, 358)",3


We found about 625 frequent itemsets with support > 0.8 and length > 2. These look good. Let's mine the association rules.

## Association Rule Mining

Let's mine association rules from the obtained frequent itemsets and take a look at rules ($R = I\rightarrow j$) with high *confidence* ($Conf$) and *interest* ($Interest$) (aka *levarage*):

$$
Conf(I\rightarrow j) = \frac{Support(I \cup j)}{Support(I)} \\
Interest(I \rightarrow j) = Conf(I\rightarrow j) - Pr [j]
$$
where $Pr [j]$ is the probability of the item $j$ computed as: $support(I) \times support(j)$

### Association Rules Hyperparameters

In [62]:
confidence = 0.80     # Association Rule Confidence
interest   = 0.22     # Association Rule Interest ()

### Association Rules
Using the association rule mining algorithms from: *Agrawal, Rakesh, and Ramakrishnan Srikant. "Fast algorithms for mining association rules." Proc. 20th int. conf. very large data bases, VLDB. Vol. 1215. 1994*. Using implementation from the [mlxtend.association_rules](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/) Python package.

In [45]:
# Get association rules based on the confidence 
rules_conf = association_rules(freq_itemsets, 
                               metric='confidence', 
                               min_threshold=confidence, 
                               support_only=False)

In [63]:
# Get association rules based on the leverage (interest)
rules_intr = association_rules(freq_itemsets, 
                               metric='leverage', 
                               min_threshold=interest, 
                               support_only=False)

### Association Rules Visualization

In [58]:
rules_conf

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(20),(26),0.216338,0.486036,0.191702,0.886123,1.823161,0.086554,4.513309
1,(20),(342),0.216338,0.744764,0.175544,0.811434,1.089519,0.014423,1.353564
2,(20),(343),0.216338,0.756134,0.178536,0.825265,1.091427,0.014956,1.395634
3,(20),(344),0.216338,0.763315,0.181428,0.838635,1.098675,0.016295,1.466768
4,(20),(345),0.216338,0.776980,0.184421,0.852467,1.097154,0.016331,1.511658
5,(20),(346),0.216338,0.783363,0.188211,0.869986,1.110578,0.018740,1.666259
6,(20),(347),0.216338,0.790844,0.190505,0.880590,1.113482,0.019415,1.751582
7,(20),(348),0.216338,0.801217,0.193198,0.893038,1.114602,0.019864,1.858451
8,(20),(349),0.216338,0.807500,0.195093,0.901798,1.116777,0.020400,1.960241
9,(20),(350),0.216338,0.810792,0.196589,0.908714,1.120773,0.021184,2.072688


In [67]:
rules_intr.sort_values(by=['leverage'])

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
380,"(369, 331, 332)",(329),0.527229,0.528426,0.498604,0.945706,1.789665,0.220002,8.685504
383,(329),"(369, 331, 332)",0.528426,0.527229,0.498604,0.943564,1.789665,0.220002,8.377056
398,"(329, 342)","(331, 333)",0.512867,0.556254,0.505286,0.985220,1.771170,0.220002,30.022938
399,"(331, 333)","(329, 342)",0.556254,0.512867,0.505286,0.908374,1.771170,0.220002,5.316523
640,"(360, 333, 334)",(331),0.580192,0.571614,0.551666,0.950834,1.663420,0.220020,8.713016
643,(331),"(360, 333, 334)",0.571614,0.580192,0.551666,0.965102,1.663420,0.220020,12.029613
31,"(326, 343)","(328, 329)",0.431079,0.455815,0.416517,0.966219,2.119763,0.220025,16.109370
30,"(328, 329)","(326, 343)",0.455815,0.431079,0.416517,0.913786,2.119763,0.220025,6.598904
232,"(328, 371)","(330, 331)",0.472970,0.511171,0.461799,0.976381,1.910088,0.220031,20.696673
233,"(330, 331)","(328, 371)",0.511171,0.472970,0.461799,0.903415,1.910088,0.220031,5.456621


### Results
So we have 750 rules of our interest (high *support, confidence* and *interest*). Let's dig deeper in these results.

## Interpretation of Results
Let's see which spectra have the *antecedants* and *consequents* peaks to get more insight into the results.

In [65]:
# Let's sort the results based on the leverage (interest)
res = rules_intr.sort_values(by=['leverage'])

In [66]:
res

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
380,"(369, 331, 332)",(329),0.527229,0.528426,0.498604,0.945706,1.789665,0.220002,8.685504
383,(329),"(369, 331, 332)",0.528426,0.527229,0.498604,0.943564,1.789665,0.220002,8.377056
398,"(329, 342)","(331, 333)",0.512867,0.556254,0.505286,0.985220,1.771170,0.220002,30.022938
399,"(331, 333)","(329, 342)",0.556254,0.512867,0.505286,0.908374,1.771170,0.220002,5.316523
640,"(360, 333, 334)",(331),0.580192,0.571614,0.551666,0.950834,1.663420,0.220020,8.713016
643,(331),"(360, 333, 334)",0.571614,0.580192,0.551666,0.965102,1.663420,0.220020,12.029613
31,"(326, 343)","(328, 329)",0.431079,0.455815,0.416517,0.966219,2.119763,0.220025,16.109370
30,"(328, 329)","(326, 343)",0.455815,0.431079,0.416517,0.913786,2.119763,0.220025,6.598904
232,"(328, 371)","(330, 331)",0.472970,0.511171,0.461799,0.976381,1.910088,0.220031,20.696673
233,"(330, 331)","(328, 371)",0.511171,0.472970,0.461799,0.903415,1.910088,0.220031,5.456621


### Results
So we have 750 rules of our interest (high *support, confidence* and *interest*). Let's dig deeper in these results.

## Interpretation of Results